In [17]:
import xarray as xr
import xesmf
import numpy as np

In [30]:
# import xesmf._version


print(xr.__version__)
print(xesmf.__version__)
print(np.__version__)

2024.10.0
0.8.8
2.0.2


In [19]:
weights_file = '/Users/brianpm/Documents/mapping_ne30pg3_to_fv09_esmfbilin.nc'
weights = xr.open_dataset(weights_file)


ds_in = xr.open_dataset("/Users/brianpm/Documents/f.e30_beta04.FLTHIST.ne30.convproc_do_aerF.001.cam.h1a.2000-01.nc")
ds_src = xr.open_dataset("/Users/brianpm/Documents/ne30pg3_scrip_170604.nc")
ds_data = xr.open_dataset("/Users/brianpm/Documents/f.e30_beta04.FLTHIST.ne30.convproc_do_aerF.001_CLDTOT_CAL_climo.nc")
ds_tgt = xr.open_dataset("/Users/brianpm/Documents/model_output/fgray001/fgray001.cam.h0a.0001-06-30-00000.nc")

In [20]:
# input variable shape
in_shape = weights.src_grid_dims.load().data

# Since xESMF expects 2D vars, we'll insert a dummy dimension of size-1
if len(in_shape) == 1:
    in_shape = [1, in_shape.item()]

# output variable shape
out_shape = weights.dst_grid_dims.load().data.tolist()[::-1]

in_shape, out_shape

([1, 48600], [192, 288])

In [21]:
dummy_in = xr.Dataset(
    {
        "lat": ("lat", np.empty((in_shape[0],))),
        "lon": ("lon", np.empty((in_shape[1],))),
    }
)
dummy_out = xr.Dataset(
    {
        "lat": ("lat", weights.yc_b.data.reshape(out_shape)[:, 0]),
        "lon": ("lon", weights.xc_b.data.reshape(out_shape)[0, :]),
    }
)

regridder = xesmf.Regridder(
    dummy_in,
    dummy_out,
    weights=weights_file,
    method="bilinear",
    reuse_weights=True,
    periodic=True,
)

In [34]:
x = ds_data['CLDTOT_CAL']

In [26]:
vars_with_ncol = [name for name in ds_data.variables if "ncol" in ds_data[name].dims]
updated = ds_data.copy().update(
    ds_data[vars_with_ncol].transpose(..., "ncol").expand_dims("dummy", axis=-2)
)
updated

<xarray.Dataset> Size: 2MB
Dimensions:     (time: 12, dummy: 1, ncol: 48600)
Coordinates:
  * time        (time) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: dummy, ncol
Data variables:
    CLDTOT_CAL  (time, dummy, ncol) float32 2MB 63.47 64.84 67.68 ... 77.7 73.14
Attributes: (12/17)
    ne:                 30
    fv_nphys:           3
    Conventions:        CF-1.0
    source:             CAM
    case:               f.e30_beta04.FLTHIST.ne30.convproc_do_aerF.001
    logname:            hannay
    ...                 ...
    adf_user:           brianpm
    hist_file_locs:     /glade/derecho/scratch/hannay/archive/f.e30_beta04.FL...
    hist_file_list:     f.e30_beta04.FLTHIST.ne30.convproc_do_aerF.001.cam.h1...
    NCO:                netCDF Operators version 5.2.6 (Homepage = http://nco...
    climo_yrs:          2000-2009
    time_series_files:  /glade/derecho/scratch/brianpm/ADF/f.e30_beta04.FLTHI...

In [27]:
regridder(updated) # This works on a dataset

/opt/homebrew/Caskroom/miniforge/base/envs/p12/lib/python3.12/site-packages/xesmf/frontend.py:718: UserWarning: Using dimensions ('dummy', 'ncol') from data variable CLDTOT_CAL as the horizontal dimensions for the regridding.
  warnings.warn(


<xarray.Dataset> Size: 3MB
Dimensions:     (time: 12, lat: 192, lon: 288)
Coordinates:
  * time        (time) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * lat         (lat) float64 2kB -90.0 -89.06 -88.12 ... 88.12 89.06 90.0
  * lon         (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 355.0 356.3 357.5 358.8
Data variables:
    CLDTOT_CAL  (time, lat, lon) float32 3MB 8.784 8.784 8.784 ... 19.28 19.28
Attributes:
    regrid_method:  bilinear

In [37]:

regridder(x.transpose(...,"ncol").expand_dims("dummy",axis=-2))

/opt/homebrew/Caskroom/miniforge/base/envs/p12/lib/python3.12/site-packages/xesmf/frontend.py:718: UserWarning: Using dimensions ('dummy', 'ncol') from data variable CLDTOT_CAL as the horizontal dimensions for the regridding.
  warnings.warn(


<xarray.DataArray (time: 12, lat: 192, lon: 288)> Size: 3MB
array([[[ 8.784274 ,  8.784274 ,  8.784274 , ...,  8.784274 ,
          8.784274 ,  8.784274 ],
        [ 7.883109 ,  7.8450346,  7.807945 , ...,  8.003364 ,
          7.962262 ,  7.922181 ],
        [ 6.372375 ,  6.327855 ,  6.283938 , ...,  6.50983  ,
          6.4633346,  6.4175253],
        ...,
        [38.22031  , 38.410076 , 38.592045 , ..., 37.610405 ,
         37.81985  , 38.02336  ],
        [33.172043 , 33.217995 , 33.262318 , ..., 33.024902 ,
         33.075443 , 33.124508 ],
        [30.686493 , 30.686493 , 30.686493 , ..., 30.686493 ,
         30.686493 , 30.686493 ]],

       [[ 7.7165174,  7.7165174,  7.7165174, ...,  7.7165174,
          7.7165174,  7.7165174],
        [ 6.1658025,  6.144904 ,  6.124469 , ...,  6.2312984,
          6.2089977,  6.187166 ],
        [ 5.124068 ,  5.1018605,  5.0806766, ...,  5.197378 ,
          5.1717725,  5.1473536],
...
        [39.251495 , 39.462723 , 39.66648  , ..., 38.575012 ,
         38.807438 , 39.033    ],
        [34.653637 , 34.721607 , 34.78828  , ..., 34.44303  ,
         34.514236 , 34.584476 ],
        [32.590626 , 32.590626 , 32.590626 , ..., 32.590626 ,
         32.590626 , 32.590626 ]],

       [[ 3.9274194,  3.9274194,  3.9274194, ...,  3.9274194,
          3.9274194,  3.9274194],
        [ 3.5306005,  3.5168092,  3.5031412, ...,  3.572765 ,
          3.5585732,  3.5445201],
        [ 3.4570866,  3.4466605,  3.4359858, ...,  3.487354 ,
          3.4773853,  3.467312 ],
        ...,
        [25.581741 , 25.665947 , 25.743423 , ..., 25.294317 ,
         25.395367 , 25.491362 ],
        [21.566704 , 21.59885  , 21.629946 , ..., 21.463495 ,
         21.499073 , 21.53346  ],
        [19.284275 , 19.284275 , 19.284275 , ..., 19.284275 ,
         19.284275 , 19.284275 ]]], dtype=float32)
Coordinates:
  * time     (time) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.3 357.5 358.8
Attributes:
    regrid_method:  bilinear

In [6]:
lat_target = ds_tgt.lat.data
lon_target = ds_tgt.lon.data
ds_out = xr.Dataset({'lat': (['lat'], lat_target), 'lon': (['lon'], lon_target)})

In [ ]:
# Define the regridder
# xESMF, internally via ESMF, will read and use the grid info from the SCRIP file
regridder = xe.Regridder(src_grid_ds, ds_out, method='bilinear', periodic=True)



In [ ]:

# Perform the regridding
regridded_data = regridder(ds_src['varname'])

# Write regridded output if needed
regridded_data.to_netcdf('regridded_output.nc')


In [ ]:
# Regrids unstructured SE grid to regular lat-lon
# Shamelessly borrowed from @maritsandstad with NorESM who deserves credit for this work
# https://github.com/NorESMhub/xesmf_clm_fates_diagnostic/blob/main/src/xesmf_clm_fates_diagnostic/plotting_methods.py

import xarray as xr
import xesmf
import numpy as np

def make_se_regridder(weight_file, s_data, d_data,
                      Method='coservative'
                      ):
    weights = xr.open_dataset(weight_file)
    in_shape = weights.src_grid_dims.load().data

    # Since xESMF expects 2D vars, we'll insert a dummy dimension of size-1
    if len(in_shape) == 1:
        in_shape = [1, in_shape.item()]

    # output variable shape
    out_shape = weights.dst_grid_dims.load().data.tolist()[::-1]

    dummy_in = xr.Dataset(
        {
            "lat": ("lat", np.empty((in_shape[0],))),
            "lon": ("lon", np.empty((in_shape[1],))),
        }
    )
    dummy_out = xr.Dataset(
        {
            "lat": ("lat", weights.yc_b.data.reshape(out_shape)[:, 0]),
            "lon": ("lon", weights.xc_b.data.reshape(out_shape)[0, :]),
        }
    )
    # Hard code masks for now, not sure this does anything?
    s_mask = xr.DataArray(s_data.data.reshape(in_shape[0],in_shape[1]), dims=("lat", "lon"))
    dummy_in['mask']= s_mask
    
    d_mask = xr.DataArray(d_data.values, dims=("lat", "lon"))  
    dummy_out['mask']= d_mask                

    # do source and destination grids need masks here?
    # See xesmf docs https://xesmf.readthedocs.io/en/stable/notebooks/Masking.html#Regridding-with-a-mask
    regridder = xesmf.Regridder(
        dummy_in,
        dummy_out,
        weights=weight_file,
        # results seem insensitive to this method choice
        # choices are coservative_normed, coservative, and bilinear
        method=Method,
        reuse_weights=True,
        periodic=True,
    )
    return regridder

def regrid_se_data_bilinear(regridder, data_to_regrid):
    updated = data_to_regrid.copy().transpose(..., "lndgrid").expand_dims("dummy", axis=-2)
    regridded = regridder(updated.rename({"dummy": "lat", "lndgrid": "lon"}),
                         skipna=True, na_thres=1,
                         )
    return regridded

def regrid_se_data_conservative(regridder, data_to_regrid):
    updated = data_to_regrid.copy().transpose(..., "lndgrid").expand_dims("dummy", axis=-2)
    regridded = regridder(updated.rename({"dummy": "lat", "lndgrid": "lon"}) )
    return regridded

In [44]:
np.arange(-105, 105
          , 5)

array([-105, -100,  -95,  -90,  -85,  -80,  -75,  -70,  -65,  -60,  -55,
        -50,  -45,  -40,  -35,  -30,  -25,  -20,  -15,  -10,   -5,    0,
          5,   10,   15,   20,   25,   30,   35,   40,   45,   50,   55,
         60,   65,   70,   75,   80,   85,   90,   95,  100])

In [41]:
np.prod

array([-100.        ,  -91.66666667,  -83.33333333,  -75.        ,
        -66.66666667,  -58.33333333,  -50.        ,  -41.66666667,
        -33.33333333,  -25.        ,  -16.66666667,   -8.33333333,
          0.        ,    8.33333333,   16.66666667,   25.        ,
         33.33333333,   41.66666667,   50.        ,   58.33333333,
         66.66666667,   75.        ,   83.33333333,   91.66666667,
        100.        ])